# MNIST Classification using Neural Nets

## Importing Libraries

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow_datasets as tfds

## Data

In [3]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True,  as_supervised = True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)


validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline Model

In [13]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

### Choosing Optimizer and Loss Function

In [14]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

### Training

In [15]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), verbose = 2)

Epoch 1/5
540/540 - 3s - 5ms/step - accuracy: 0.9241 - loss: 0.2631 - val_accuracy: 0.9593 - val_loss: 0.1367
Epoch 2/5
540/540 - 2s - 3ms/step - accuracy: 0.9693 - loss: 0.1013 - val_accuracy: 0.9757 - val_loss: 0.0819
Epoch 3/5
540/540 - 2s - 4ms/step - accuracy: 0.9793 - loss: 0.0684 - val_accuracy: 0.9773 - val_loss: 0.0724
Epoch 4/5
540/540 - 2s - 4ms/step - accuracy: 0.9824 - loss: 0.0553 - val_accuracy: 0.9830 - val_loss: 0.0560
Epoch 5/5
540/540 - 2s - 4ms/step - accuracy: 0.9865 - loss: 0.0428 - val_accuracy: 0.9872 - val_loss: 0.0381


### Testing

In [22]:
test_loss, test_accuracy = model.evaluate(test_data)

print('Test Loss = {:.2f}. Test Accuracy = {:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - accuracy: 0.9775 - loss: 0.0776
Test Loss = 0.08. Test Accuracy = 97.75%
